#Setup

In [10]:
import math
import random

import torch
from torch import nn, Tensor
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.nn.parameter import Parameter, UninitializedParameter
import torch.nn.functional as F
from torch.nn.modules.module import Module
from torch.nn import init
from tqdm.notebook import trange, tqdm
import torchvision.transforms.functional as TF
from typing import Optional, List, Tuple, Union
from torch.nn.common_types import _size_1_t, _size_2_t, _size_3_t
import itertools
import numpy as np

In [11]:
dataset = 'FashionMNIST'
datadir = 'datasets'
arch = 'hhnmlpb'
batchsize = 64
epochs = 50
nlayers = 1
width = 32
lr = 0.001
dimensions = 5
hin = 2
nchannels = 1
nclasses = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(root="data", train=True, download=True, 
                                      transform=transforms.Compose([transforms.Resize((16, 16)),
                                                                    transforms.ToTensor()]),)

# Download test data from open datasets
test_data = datasets.FashionMNIST(root="data", train=False, download=True,
                                  transform=transforms.Compose([transforms.Resize((16, 16)),
                                                                    transforms.ToTensor()]),)

In [13]:
# Create data loaders
train_loader = DataLoader(training_data, batch_size=batchsize, shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=batchsize, shuffle=True, num_workers=8, pin_memory=True)

for X, y in test_loader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 16, 16])
Shape of y:  torch.Size([64]) torch.int64


# MLP with search in alpha-space



In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def count_model_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class HHN_MLPB(nn.Module):
    def __init__(self, hin, dimensions, n_layers, n_units, n_channels, n_classes=10):
        super(HHN_MLPB, self).__init__()
        self.hyper_stack = nn.Sequential(
            nn.Linear(hin, 64),
            nn.ReLU(),
            nn.Linear(64, dimensions),
            nn.Softmax(dim=0)
        )

        self.dimensions = dimensions
        self.n_layers = n_layers
        self.n_units = n_units
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        self.bns = []

        self.weight_list_fc1, self.bias_list_fc1 = \
            self.create_param_combination_linear(in_features=16 * 16 * n_channels, out_features=n_units)
        self.weights = nn.ParameterList()
        self.biases = nn.ParameterList()
        for _ in range(n_layers - 1):
            w, b = self.create_param_combination_linear(in_features=n_units, out_features=n_units)
            self.weights += w
            self.biases += b
            self.bns.append(nn.BatchNorm1d(self.n_units).to(self.device))
        self.weight_list_fc2, self.bias_list_fc2 = self.create_param_combination_linear(in_features=n_units,
                                                                                        out_features=n_classes)

    def create_param_combination_linear(self, in_features, out_features):
        weight_list = nn.ParameterList()
        bias_list = nn.ParameterList()
        for _ in range(self.dimensions):
            weight = Parameter(torch.empty((out_features, in_features)))
            nn.init.kaiming_uniform_(weight, a=math.sqrt(5))
            weight_list.append(weight)

            bias = Parameter(torch.empty(out_features))
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(weight)
            bound = 1 / math.sqrt(fan_in)
            nn.init.uniform_(bias, -bound, bound)
            bias_list.append(bias)
        return weight_list, bias_list

    def calculate_weighted_sum(self, param_list: List, factors: Tensor):
        weighted_list = [a * b for a, b in zip(param_list, factors)]
        return torch.sum(torch.stack(weighted_list), dim=0)

    def forward(self, x, hyper_x):
        hyper_output = self.hyper_stack(hyper_x)

        weight_fc1 = self.calculate_weighted_sum(self.weight_list_fc1, hyper_output)
        weight_fc2 = self.calculate_weighted_sum(self.weight_list_fc2, hyper_output)

        bias_fc1 = self.calculate_weighted_sum(self.bias_list_fc1, hyper_output)
        bias_fc2 = self.calculate_weighted_sum(self.bias_list_fc2, hyper_output)

        logits = torch.flatten(x, start_dim=1)
        logits = F.linear(logits, weight=weight_fc1, bias=bias_fc1)
        logits = torch.relu(logits)

        it_w = iter(self.weights)
        it_b = iter(self.biases)
        for (w, b, bn) in zip(zip(*[it_w] * self.dimensions), zip(*[it_b] * self.dimensions), self.bns):
            w = nn.ParameterList(w)
            b = nn.ParameterList(b)
            w = self.calculate_weighted_sum(w.to(self.device), hyper_output)
            b = self.calculate_weighted_sum(b.to(self.device), hyper_output)
            logits = F.linear(logits, weight=w, bias=b)
            logits = bn(logits)
            logits = torch.relu(logits)
        logits = F.linear(logits, weight=weight_fc2, bias=bias_fc2)
        return logits

model = HHN_MLPB(2, dimensions, nlayers, width, 1, n_classes=10).to(device)
print(model)
print(count_model_parameters(model))

HHN_MLPB(
  (hyper_stack): Sequential(
    (0): Linear(in_features=2, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=5, bias=True)
    (3): Softmax(dim=0)
  )
  (weight_list_fc1): ParameterList(
      (0): Parameter containing: [torch.float32 of size 32x256 (GPU 0)]
      (1): Parameter containing: [torch.float32 of size 32x256 (GPU 0)]
      (2): Parameter containing: [torch.float32 of size 32x256 (GPU 0)]
      (3): Parameter containing: [torch.float32 of size 32x256 (GPU 0)]
      (4): Parameter containing: [torch.float32 of size 32x256 (GPU 0)]
  )
  (bias_list_fc1): ParameterList(
      (0): Parameter containing: [torch.float32 of size 32 (GPU 0)]
      (1): Parameter containing: [torch.float32 of size 32 (GPU 0)]
      (2): Parameter containing: [torch.float32 of size 32 (GPU 0)]
      (3): Parameter containing: [torch.float32 of size 32 (GPU 0)]
      (4): Parameter containing: [torch.float32 of size 32 (GPU 0)]
  )
  (weights): Paramete

In [15]:
def transform_angle(angle):
    cos = math.cos(angle / 180 * math.pi)
    sin = math.sin(angle / 180 * math.pi)
    return Tensor([cos, sin])

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs, eta_min=0)

cos = nn.CosineSimilarity(dim=0, eps=1e-6)

def train(dataloader, model, loss_fn, optimizer):
    for batch, (X, y) in enumerate(tqdm(dataloader, desc='Training')):
        X, y = X.to(device), y.to(device)
        angle = random.uniform(0, 360)
        X = TF.rotate(X, angle)

        # make prediction and minimize loss
        pred = model(X, hyper_x=transform_angle(angle).to(device))
        loss = loss_fn(pred, y)

        # regularizer (cossim squared) in the beta space
        beta1 = model.hyper_stack(transform_angle(angle).to(device))
        angle2 = random.uniform(0, 360)
        beta2 = model.hyper_stack(transform_angle(angle2).to(device))
        loss += pow(cos(beta1, beta2),2)

        # minimize entropy to the correct degree
        b = (F.softmax(pred, dim=1)) * (-1 * F.log_softmax(pred, dim=1))
        loss += 0.01*b.sum()

        # maximize entropy to the wrong degree
        logits = model(X, hyper_x=transform_angle(angle2).to(device))
        b2 = (F.softmax(logits, dim=1)) * (-1 * F.log_softmax(logits, dim=1))
        loss -= 0.01*b2.sum()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # scheduler.step()

def validate(dataloader, model, loss_fn):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            angle = random.uniform(0, 360)
            X = TF.rotate(X, angle)

            pred = model(X, hyper_x=transform_angle(angle).to(device))
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= len(dataloader)
    correct /= len(dataloader.dataset)
    print(f"Test with angle={angle}: Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
    return correct, test_loss

for t in range(epochs):
    print(f"=================\n Epoch: {t + 1} \n=================")
    train(train_loader, model, loss_fn, optimizer)
    test_acc, test_loss = validate(test_loader, model, loss_fn)
print("Done!")

 Epoch: 1 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

Test with angle=204.7432889822147: Accuracy: 65.8%, Avg loss: 0.920305
 Epoch: 2 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

Test with angle=24.165027934953788: Accuracy: 69.6%, Avg loss: 0.822181
 Epoch: 3 


...

Test with angle=123.49425052527964: Accuracy: 79.1%, Avg loss: 0.604402
 Epoch: 49 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

Test with angle=133.3983321527365: Accuracy: 79.4%, Avg loss: 0.594718
 Epoch: 50 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

Test with angle=116.93875613785582: Accuracy: 79.5%, Avg loss: 0.594277
Done!


Optimize in the alpha space

In [17]:
# model to eval mode and move to cpu
model.eval()
model.cpu()

# freeze Ws
for param in model.parameters():
  param.requires_grad = False

In [18]:
# execute only if you wish to test with a different batch_size (for example, batch_size=1 ... takes long!)
batch_size=64
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)

In [19]:
from scipy import optimize

# function to minimize by the basin hopping algorithm
def f(z, *args):
    alpha = transform_angle(((1+z)*180)%360-180)
    X = args[0]
    logits = model(Tensor(X), hyper_x=Tensor(alpha))
    b = (F.softmax(logits, dim=1)) * (-1 * F.log_softmax(logits, dim=1))  # entropy
    return b.sum().numpy()

# given a batch of images find the rotation angle alpha
def findalpha(X):
    # Basin hopping algorithm
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.basinhopping.html
    minimizer_kwargs = {"method": "BFGS", "args":X}
    res = optimize.basinhopping(f, 0.0, minimizer_kwargs=minimizer_kwargs, niter=100, T=5)

    alpha = ((1+res.x[0])*180)%360-180    
    # print("alpha estimate = ", alpha)     # obtained minimum
    # print("fun = ", res.fun)              # function value at minimum
    return alpha

result = 0.0
for _, (X, y) in enumerate(tqdm(test_loader, desc='Testing alpha search')):
    angle = random.uniform(-180, 180)
    # print("=============")
    # print("alpha true = ", angle)
    X = TF.rotate(X, angle)

    alpha = findalpha(X)

    # compute model prediction with the estimated alpha
    logits = model(X, hyper_x=transform_angle(alpha))
    # y is the true label --> calculate accuracy
    correct = (logits.argmax(1) == y).type(torch.float).sum().item() / batch_size
    # print(f"accuracy = {(100*correct):>0.1f}")
    result += correct

result /= len(test_loader.dataset) / batch_size
print(f"Test accuracy: {(100*result):>0.1f}%")

print("Done!")

Testing alpha search:   0%|          | 0/157 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Test accuracy: 79.7%
Done!


# One4All

The code below is to compare the performance of SCN and SCN+findalpha() to One4All.

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MLPB(nn.Module):
    def __init__(self, n_layers, n_units, n_channels, n_classes=10):
        super(MLPB, self).__init__()
        mid_layers = []
        mid_layers.extend([nn.Flatten(), nn.Linear(16 * 16 * n_channels, n_units), nn.ReLU()])
        for _ in range(n_layers-1):
            mid_layers.extend([
                nn.Linear(n_units, n_units),
                nn.BatchNorm1d(n_units),
                nn.ReLU(),
            ])
        mid_layers.extend([nn.Linear(n_units, n_classes)])
        self.linear_relu_stack = nn.Sequential(*mid_layers)

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model_one4all = MLPB(nlayers, width, 1, n_classes=10).to(device)
print(model_one4all)
print(count_model_parameters(model_one4all))
model_one4all.to(device)

MLPB(
  (linear_relu_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=256, out_features=32, bias=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=10, bias=True)
  )
)
8554


MLPB(
  (linear_relu_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=256, out_features=32, bias=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=10, bias=True)
  )
)

In [21]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_one4all.parameters())
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs, eta_min=0)

def train(dataloader, model_one4all, loss_fn, optimizer):
    for batch, (X, y) in enumerate(tqdm(dataloader, desc='Training')):
        X, y = X.to(device), y.to(device)
        angle = random.uniform(0, 360)
        X = TF.rotate(X, angle)

        pred = model_one4all(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # scheduler.step()

def validate(dataloader, model_one4all, loss_fn):
    model_one4all.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            angle = random.uniform(0, 360)
            X = TF.rotate(X, angle)

            pred = model_one4all(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= len(dataloader)
    correct /= len(dataloader.dataset)
    print(f"Test with translation={angle}: Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
    return correct, test_loss

for t in range(epochs):
    print(f"=================\n Epoch: {t + 1} \n=================")
    train(train_loader, model_one4all, loss_fn, optimizer)
    test_acc, test_loss = validate(test_loader, model_one4all, loss_fn)
print("Done!")

 Epoch: 1 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Test with translation=148.6699231021879: Accuracy: 56.8%, Avg loss: 1.246317
 Epoch: 2 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

...

Test with translation=173.41471443397316: Accuracy: 70.0%, Avg loss: 0.835227
 Epoch: 49 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

Test with translation=201.4077664315325: Accuracy: 69.9%, Avg loss: 0.846756
 Epoch: 50 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

Test with translation=246.5534410344573: Accuracy: 69.7%, Avg loss: 0.839129
Done!
